In [ ]:
def main():
    try:    
        import re
        import time
        import requests
        import pandas as pd
        from bs4 import BeautifulSoup
        filename = "updated_blikkin_flipkart"
        data = pd.read_csv(filename+'.csv',index_col=0)
        print("data shape:",data.shape)
        # cols = ['total']
        total = data['total']
        data_url = []
        data_code = []
        for string in total:
            url_match = re.search(r"'(https://www\.flipkart\.com/[^']+)'", string)
            url = url_match.group(1) if url_match else None
            data_url.append(url)
            words = string.split("}")[1].split(", '")[1].split("', [")[0]
            data_code.append(words)
        data_dict = dict(zip(data_code,data_url))
        review_pages_dictionary = dict()
        for dc,dl in data_dict.items():
            if dl!=None:
                review_pages_list = []
                review_pages_list.append(dl)
                i=2
                while i<=10:
                    nextLink = dl+f"&page={i}"
                    #print(nextLink)
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
                    response = requests.get(nextLink, headers=headers)
                    #print(response,response.ok)
                    if response.ok:
                        html_soup = BeautifulSoup(response.text, 'html.parser')
                        review = html_soup.find_all('div', {"class":"col _2wzgFH K0kLPL _1QgsS5"})
                        if review:
                            review_pages_list.append(nextLink)               
                    i=i+1
            review_pages_dictionary[dc]=review_pages_list
        codeList=[]
        rating=[]
        reviews=[]
        name=[]
        dtime=[]
        ifCertified=[]
        for dc,dl in review_pages_dictionary.items():
            if dl!=None:
                for l in dl:
                    if l!=None:
                        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
                        response = requests.get(l, headers=headers)
                        if response.ok:
                            html_soup = BeautifulSoup(response.text, 'html.parser')
                            review = html_soup.find_all('div', {"class":"col _2wzgFH K0kLPL _1QgsS5"})
                            if review:
                                for rev in review:
                                    codeList.append(dc)
                                    rat=rev.find("div",{"class":"_3LWZlK _1BLPMq _3B8WaH"})
                                    rating.append(rat.get_text() if rat else 0)
                                    revw=rev.find("div",{"class":"_6K-7Co"})
                                    reviews.append(revw.get_text() if revw else '=')
                                    n=rev.find("p",{"class":"_2sc7ZR _2V5EHH _1QgsS5"})
                                    name.append(n.get_text() if n else "=")
                                    my=rev.find("div",{"class":"row _1ExUpQ"}).find_all("p",{"class":"_2sc7ZR"})[1].get_text()
                                    dtime.append(my if my else 0)
                                    ic=rev.find("p",{"class":"_2mcZGG"})
                                    ifCertified.append(ic.get_text() if ic else "-")
                        else:
                            print("Timeout !!!!")
        final_dataframe = pd.DataFrame({
            'Code': codeList,
            'Rating': rating,
            'Review': reviews,
            'Name': name,
            'Time': dtime,
            'Certified': ifCertified
        })
        final_dataframe.to_csv("Reviews_ALL_"+filename+".csv", index=False)
    except Exception as e:
        print("\nError !!!!!!!", e)

In [ ]:
if __name__ == "__main__":
    main()